In [1]:
import Base: iterate, exp, log, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,max, isless
using BenchmarkTools

In [2]:
# The elements are `value`, the result of a computation; 
# `dfdy` a storage location where the derivative of some overall 
# computation (`f(value)`) with respect to the output of this element is stored;

# `parent` storing the "parent" locations that have been used to produce the result `value`;
# `backpropagation!` a function that propogates the derivative `dfdy` to the
# parents via `dfdx = dydx * dfdy` where `dydx` is the derivative of this piece
# of the computation, `dfdy` is the derivative of the overall function with
# respect to the output, and `dfdx` is the same for the next level up.
mutable struct Dual{T <: Number} <: Number
    value::T
    dfdy::T
    parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}
    backpropagation!
end
# Dual(n::Integer, d::Float64, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)
# Dual(n::Float64, d::Integer, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)


In [3]:
# x = Dual(2.0, 2, 1, (dfdy, parents) -> nothing)*  Dual(2, 2, 1, (dfdy, parents) -> nothing)
# promote_type(Dual{Real}, Dual{Number})

In [4]:
function convert(::Type{Dual{T}}, x::T) where T <: Number
    Dual(x, zero(T), nothing, (dfdy, parents) -> nothing)
end

# function convert(::Type{Dual{T}}, x::Dual{S}) where {T, S <: Number}
#     Dual(T(x.value), T(x.dfdy), x.parent, x.backpropagation!)
# end


function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
#     print("2asdsadsda", typeof(x), "\n")
    Dual(T(x), zero(T), nothing, (dfdy, parents) -> nothing)
end

function convert(::Type{Dual{T}}, x::Dual{T}) where T
    x
end


convert (generic function with 186 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(T), zero(T), nothing, (dfdy, parents) -> nothing)
end

zero (generic function with 23 methods)

In [6]:
# function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
#     Dual{promote_type(T,S)}
# end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S, T)}
end


promote_rule (generic function with 125 methods)

In [7]:
function push_parents!(queue::Array{Dual{T}, 1}, ::Nothing) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, i::Int) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, ls::Array{Dual{T}, 1}) where T
    append!(queue, ls)
end
function push_parents!(queue::Array{Dual{T}, 1}, l::Dual{T}) where T
    push!(queue, l)
end

push_parents! (generic function with 4 methods)

In [8]:
function backprop!(l::Dual{T}) where T
    # Apparently we need this construction because otherwise l gets copied when
    # put into the array.
    backprop!([l])
end
function backprop!(queue::Array{Dual{T},1}) where T
    while length(queue) > 0
        l = popfirst!(queue)
        l.backpropagation!(l.dfdy, l.parent)
        push_parents!(queue, l.parent)
    end
end

backprop! (generic function with 2 methods)

In [9]:
function collect_outputs(l::Dual{T}) where T
    queue = Dual{T}[l]

    outputs = Dual{T}[]

    while length(queue) > 0
        l = popfirst!(queue)
        if typeof(l.parent) <: Int
            push!(outputs, l)
        elseif typeof(l.parent) == Dual{T}
            push!(queue, l.parent)
        elseif typeof(l.parent) == Array{Dual{T}, 1}
            append!(queue, l.parent)
        else # Nothing
            # Do nothing
        end
    end

    outputs
end


collect_outputs (generic function with 1 method)

In [10]:
function derivativeСalculation(f)
    function dfdx(x::T) where T <: Number
        # Pass the function a backward infinitesimal whose backprop function
        # stores the backprop derivative in dfdx_store
#         print("x::T", "\n")

        x = Dual(x, zero(x), 1, (dfdy, parents) -> nothing)

        result = f(x)

        result.dfdy = one(result.value)
        backprop!(result)

        y = collect_outputs(result)[1]

        return y.dfdy
    end

    function dfdx(x::Array{T}, i) where T <: Number
#         print("array", "\n")
        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs)[i]
#         println(result)
        result.dfdy = one(result.value)
        println(result.parent)
        backprop!(result)
        y = collect_outputs(result)
        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    function dfdx(x...)
        print("x...", "\n")

        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs...)
        result.dfdy = one(result.value)
        backprop!(result)
        y = collect_outputs(result)

        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    return dfdx
end

derivativeСalculation (generic function with 1 method)

In [11]:
function derivativeСalculation(i::Integer, f)
    df = derivativeСalculation(f)
    function df_wrapper(x...)
        g = df(x...)
        return g[i]
    end
    return df_wrapper
end

derivativeСalculation (generic function with 2 methods)

In [12]:
# overloading Plus
function backpropagationForPlus!(dfdy, xy)
    x, y = xy
    x.dfdy += dfdy
    y.dfdy += dfdy
end

function +(x::Dual{T}, y::Dual{T}) where T
#     println("Przecieążenie +")
    if x == y
#         println("x = y +")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value + y.value, zero(T), [x, new_y], backpropagationForPlus!)
    end
    Dual(x.value + y.value, zero(T), [x, y], backpropagationForPlus!)
end

#--------------- overloading Minus
function backpropagationForMinus!(dfdy, xy) 
    x, y = xy
    x.dfdy += dfdy
    y.dfdy -= dfdy
end

function -(x::Dual{T}, y::Dual{T}) where T
    Dual(x.value - y.value, zero(T), [x, y], backpropagationForMinus!)
end



function backpropagationForMinusOneDual!(dfdy, x)
    x.dfdy -= dfdy
end

function -(x::Dual{T}) where T
    Dual(-x.value, zero(T), x, backpropagationForMinusOneDual!)
end


#------------------- overloading Multiplication

function backpropagationForMultiplication!(dfdy, xy)
    x,y = xy
    
    x.dfdy += dfdy*y.value
    y.dfdy += x.value*dfdy
end


function *(x::Dual{T}, y::Dual{T}) where T
#     println("mnożenie", x == y)
#     println(x)
#     println(y)
    return Dual(x.value*y.value, zero(T), [x,y], backpropagationForMultiplication!)
end


#-------------------- overloading Dividing

function /(x::Dual{T}, y::Dual{T}) where T
    yinv = one(T)/y.value

    function backpropagationForDividing!(dfdy, xy)
        a,b = xy
        a.dfdy += dfdy*yinv
        b.dfdy -= a.value*dfdy*(yinv*yinv)
    end
    if x == y
#         println("x = y /")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value*yinv, zero(T), [x, new_y], backpropagationForDividing!)
    end
    Dual(x.value*yinv, zero(T), [x,y], backpropagationForDividing!)
end

#-------------------- overloading EXP

function exp(x::Dual{T}) where T
    expValue = exp(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy*expValue
    end

    Dual(expValue, zero(expValue), x, bp!)
end

function exp(xs::Array)
    return [exp(x) for x in xs]
end


#-------------------- overloading SIN
function sin(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += cos(x.value)*dfdy
    end

    sx = sin(x.value)
    Dual(sx, zero(sx), x, bp!)
end

#-------------------- overloading COS
function cos(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy -= sin(x.value)*dfdy
    end

    cx = cos(x.value)
    Dual(cx, zero(cx), x, bp!)
end
#-------------------- overloading TAN
function tan(x::Dual{T}) where T
    c = cos(x.value)
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/(c*c)
    end

    tx = tan(x.value)
    Dual(tx, zero(tx), x, bp!)
end

#---------------------- overloading SQRT
function sqrt(x::Dual{T}) where T
    sqrtValue = sqrt(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy/(2*sqrtValue)
    end

    Dual(sqrtValue, zero(sqrtValue), x, bp!)
end

#------------------ overloading ^
function ^(a::Dual{T}, x::Dual{T}) where T
    value = a.value^x.value
#     println("potegowanie daszek")
    function bp!(dfdy, xy)
        a,x = xy
#         print(typeof(dfdy * (x.value) * (a.value)^(x.value - 1)), " nic  ", typeof(dfdy * a.value^x.value*log(a.value)))
        a.dfdy += dfdy * (x.value) * (a.value)^(x.value - 1)
        x.dfdy += dfdy * a.value^x.value*log(a.value)
    end

    Dual(value, zero(value), [a, x], bp!)
end

function ^(x::Dual{T}, n::Integer) where T
    value = x.value^n
#     println("potegowanie daszek22")
    function bp!(dfdy, x)
        x.dfdy += n * dfdy*x.value^(n-1)
    end
    
    Dual(value, zero(value), x, bp!)
end

#------------------ overloading log
function log(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/x.value
    end
    
    Dual(log(x.value), zero(T), x, bp!)
end
#----------------------- overloading max
function max(a, x::Dual{T}) where T
    
    function bp!(dfdy,p)
        p.dfdy += x.value < a ? a : 1 * dfdy
    end
    
    Dual(max(0, x.value), zero(T), x, bp!)
    
end


isless(x::Dual, y::Dual) = x.value < y.value;

In [13]:

# function iterate(iter::Dual, state=1)
# #     print("iterate")
#     if state > length(iter.x)
#         return nothing
#     end
#     return (iter[state],state+1)
# end

function softmax(vector::Array)
    e = exp(vector)
#     e = [MathConstants.e^(x) for x in vector]
#     s = zero(vector[1])
#     z = [Dual(2.0, 1.0, nothing, (dfdy, parents) -> nothing), Dual(21.0, 1.0, nothing, (dfdy, parents) -> nothing)]
#     for x in z
#        s = s + x 
#     end    
#     println(s, sum(e))
    return e / sum(e)
end

softmax (generic function with 1 method)

In [14]:
J = function jacobian(f, number_of_functions, args::Vector{T}) where {T <:Number}
    jacobian_rows = Matrix{T}[]
    d = derivativeСalculation(f)  
    for i=1:number_of_functions
#         x = Dual{T}[]
#         for j=1:length(args)
#             seed = (i == j)
#             push!(x, seed ?
#                 Dual(args[j], one(args[j])) :
#                 Dual(args[j],zero(args[j])) )
#         end
#         temp  = [f(x)..]
        rows = d(args,i)
        push!(jacobian_rows, rows[:,:])
    end
    jacobian_rows
end

jacobian (generic function with 1 method)

In [15]:
function test(a, x)
    return a*x
end

test (generic function with 1 method)

In [16]:
d = derivativeСalculation(test)

(::var"#dfdx#12"{typeof(test)}) (generic function with 3 methods)

In [17]:
d(2, 3)

x...


2-element Array{Int64,1}:
 3
 2

In [18]:
# f(x::Vector) = [2x[1]*x[2], 3x[2]*x[3]^2]
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]


f (generic function with 1 method)

In [19]:
# f(x::Vector) = x[1]*x[2]*x[3]
d = derivativeСalculation(f)
d([1.0,2.0,3.0], 5)

Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForPlus!), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForPlus!)]


3-element Array{Float64,1}:
  0.37753914284153406
 -0.0755078285683068
 -0.0755078285683068

In [20]:
J(f,5,[1.,2., 3.]) 


1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(16.0, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(4.0, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(3.0, 0.0, 3, var"#9#15"()), Dual{Float64}(0.8414709848078965, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(1.0, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"()

5-element Array{Array{Float64,2},1}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -0.5555555555555556]
 [0.0; 16.0; -2.0]
 [1.6209069176044193; 0.0; 0.8414709848078965]
 [0.37753914284153406; -0.0755078285683068; -0.0755078285683068]

In [21]:
J(softmax,2,[1., 2.]) 

Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(10.107337927389695, 0.0, Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065))], backpropagationForPlus!)]
Dual{Float64}[Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065)), Dual{Float64}(10.107337927389695, 0.0, Dual{Float64}[Dual{Float64}(2.718281828459045, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.718281828459045)), Dual{Float64}(7.38905609893065, 0.0, Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), var"#bp!#20"{Float64}(7.38905609893065))], backpropagationForPlus!)]


2-element Array{Array{Float64,2},1}:
 [0.46555335461147695; -0.19661193324148185]
 [-0.19661193324148188; 0.9276705118714866]

In [22]:
x = [Dual(1.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(3.0, 0.0, nothing, nothing)]
y = [Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing)]


3-element Array{Dual{Float64},1}:
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)

In [23]:
sum(x)


Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, nothing), Dual{Float64}(2.0, 0.0, nothing, nothing)], backpropagationForPlus!), Dual{Float64}(3.0, 0.0, nothing, nothing)], backpropagationForPlus!)

In [24]:
sa(x) = sin.(x.*x)


sa (generic function with 1 method)

In [25]:
sa([2, 3])

2-element Array{Float64,1}:
 -0.7568024953079282
  0.4121184852417566

In [26]:
#J(s,3, [1., 2., 3.]) 
d = derivativeСalculation(sa)
J(sa,3, [1., 2., 3.]) 

Dual{Float64}(1.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(1.0, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)
Dual{Float64}(4.0, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, 2, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForMultiplication!)
Dual{Float64}(9.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, 3, var"#9#15"()), Dual{Float64}(3.0, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)


3-element Array{Array{Float64,2},1}:
 [1.0806046117362795; 0.0; 0.0]
 [0.0; -2.6145744834544478; 0.0]
 [0.0; 0.0; -5.466781571308061]

In [27]:
softmax([1,2])


2-element Array{Float64,1}:
 0.2689414213699951
 0.7310585786300049

In [28]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-x[1])^2 + 100*(x[2] - x[1]^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

rosenbrock (generic function with 1 method)

In [29]:
rosenbrock([1.0,2.0])

100.0

In [30]:
d = D(rosenbrock)

LoadError: [91mUndefVarError: D not defined[39m

In [31]:
@benchmark J(rosenbrock,1,[rand(1000,1)...]) 

Dual{Float64}[Dual{Float64}(19619.033627995756, 0.0, Dual{Float64}[Dual{Float64}(19618.5223852886, 0.0, Dual{Float64}[Dual{Float64}(19617.134141952574, 0.0, Dual{Float64}[Dual{Float64}(19612.570148683364, 0.0, Dual{Float64}[Dual{Float64}(19610.842753338497, 0.0, Dual{Float64}[Dual{Float64}(19594.62830924866, 0.0, Dual{Float64}[Dual{Float64}(19591.347972894793, 0.0, Dual{Float64}[Dual{Float64}(19544.17818772615, 0.0, Dual{Float64}[Dual{Float64}(19541.416206462163, 0.0, Dual{Float64}[Dual{Float64}(19525.63373425135, 0.0, Dual{Float64}[Dual{Float64}(19496.405932211863, 0.0, Dual{Float64}[Dual{Float64}(19409.649002712624, 0.0, Dual{Float64}[Dual{Float64}(19405.06940545168, 0.0, Dual{Float64}[Dual{Float64}(19394.61983721996, 0.0, Dual{Float64}[Dual{Float64}(19381.856278829506, 0.0, Dual{Float64}[Dual{Float64}(19362.598999426522, 0.0, Dual{Float64}[Dual{Float64}(19275.018471822066, 0.0, Dual{Float64}[Dual{Float64}(19252.326564870542, 0.0, Dual{Float64}[Dual{Float64}(19219.340741284115, 0.0, 

, 0.0, Dual{Float64}[Dual{Float64}(15845.907876447402, 0.0, Dual{Float64}[Dual{Float64}(15804.56729223218, 0.0, Dual{Float64}[Dual{Float64}(15797.749729871417, 0.0, Dual{Float64}[Dual{Float64}(15752.295043311775, 0.0, Dual{Float64}[Dual{Float64}(15752.196190192937, 0.0, Dual{Float64}[Dual{Float64}(15707.587642834735, 0.0, Dual{Float64}[Dual{Float64}(15705.936772884252, 0.0, Dual{Float64}[Dual{Float64}(15685.917881593547, 0.0, Dual{Float64}[Dual{Float64}(15685.86827158905, 0.0, Dual{Float64}[Dual{Float64}(15637.289677030401, 0.0, Dual{Float64}[Dual{Float64}(15631.909658817984, 0.0, Dual{Float64}[Dual{Float64}(15612.802558007232, 0.0, Dual{Float64}[Dual{Float64}(15612.56055625416, 0.0, Dual{Float64}[Dual{Float64}(15611.822045138044, 0.0, Dual{Float64}[Dual{Float64}(15567.258116548051, 0.0, Dual{Float64}[Dual{Float64}(15564.809448824535, 0.0, Dual{Float64}[Dual{Float64}(15563.977844378658, 0.0, Dual{Float64}[Dual{Float64}(15559.853626688106, 0.0, Dual{Float64}[Dual{Float64}(15542.80018131

, 0.0, Dual{Float64}[Dual{Float64}(11726.740466072395, 0.0, Dual{Float64}[Dual{Float64}(11718.517906404892, 0.0, Dual{Float64}[Dual{Float64}(11712.866985475997, 0.0, Dual{Float64}[Dual{Float64}(11636.735466899649, 0.0, Dual{Float64}[Dual{Float64}(11629.088795121806, 0.0, Dual{Float64}[Dual{Float64}(11628.685660155688, 0.0, Dual{Float64}[Dual{Float64}(11628.290626880098, 0.0, Dual{Float64}[Dual{Float64}(11628.122453117323, 0.0, Dual{Float64}[Dual{Float64}(11613.965451145516, 0.0, Dual{Float64}[Dual{Float64}(11581.872867128439, 0.0, Dual{Float64}[Dual{Float64}(11577.56253697983, 0.0, Dual{Float64}[Dual{Float64}(11528.654190710551, 0.0, Dual{Float64}[Dual{Float64}(11501.198267659609, 0.0, Dual{Float64}[Dual{Float64}(11446.881833827598, 0.0, Dual{Float64}[Dual{Float64}(11444.20779827927, 0.0, Dual{Float64}[Dual{Float64}(11435.339100322271, 0.0, Dual{Float64}[Dual{Float64}(11410.948659205129, 0.0, Dual{Float64}[Dual{Float64}(11408.813956417851, 0.0, Dual{Float64}[Dual{Float64}(11407.4987152

}(8428.131263923326, 0.0, Dual{Float64}[Dual{Float64}(8420.952336882146, 0.0, Dual{Float64}[Dual{Float64}(8348.342124194369, 0.0, Dual{Float64}[Dual{Float64}(8303.662717747344, 0.0, Dual{Float64}[Dual{Float64}(8302.166784106294, 0.0, Dual{Float64}[Dual{Float64}(8206.195147103628, 0.0, Dual{Float64}[Dual{Float64}(8198.240547250558, 0.0, Dual{Float64}[Dual{Float64}(8198.01223034655, 0.0, Dual{Float64}[Dual{Float64}(8144.563807529349, 0.0, Dual{Float64}[Dual{Float64}(8138.555678188755, 0.0, Dual{Float64}[Dual{Float64}(8106.827513293874, 0.0, Dual{Float64}[Dual{Float64}(8092.751489234129, 0.0, Dual{Float64}[Dual{Float64}(8085.454805714494, 0.0, Dual{Float64}[Dual{Float64}(8078.046710329944, 0.0, Dual{Float64}[Dual{Float64}(7990.271762890706, 0.0, Dual{Float64}[Dual{Float64}(7985.294700908915, 0.0, Dual{Float64}[Dual{Float64}(7984.2414325418795, 0.0, Dual{Float64}[Dual{Float64}(7981.0756872051215, 0.0, Dual{Float64}[Dual{Float64}(7976.415111629824, 0.0, Dual{Float64}[Dual{Float64}(7976.1379

(4891.014687155864, 0.0, Dual{Float64}[Dual{Float64}(4841.658687024086, 0.0, Dual{Float64}[Dual{Float64}(4832.664455348055, 0.0, Dual{Float64}[Dual{Float64}(4760.373538545128, 0.0, Dual{Float64}[Dual{Float64}(4758.823437435152, 0.0, Dual{Float64}[Dual{Float64}(4756.609377456309, 0.0, Dual{Float64}[Dual{Float64}(4722.232241515089, 0.0, Dual{Float64}[Dual{Float64}(4706.376001144741, 0.0, Dual{Float64}[Dual{Float64}(4703.959769346426, 0.0, Dual{Float64}[Dual{Float64}(4680.937455062127, 0.0, Dual{Float64}[Dual{Float64}(4656.2907775487165, 0.0, Dual{Float64}[Dual{Float64}(4634.514554132402, 0.0, Dual{Float64}[Dual{Float64}(4613.155889026242, 0.0, Dual{Float64}[Dual{Float64}(4598.034252918603, 0.0, Dual{Float64}[Dual{Float64}(4538.927436643459, 0.0, Dual{Float64}[Dual{Float64}(4515.809275369069, 0.0, Dual{Float64}[Dual{Float64}(4469.252633922058, 0.0, Dual{Float64}[Dual{Float64}(4445.8757251759225, 0.0, Dual{Float64}[Dual{Float64}(4438.472522103517, 0.0, Dual{Float64}[Dual{Float64}(4397.4773

...

Dual{Float64}(0.3287017602553073, 0.0, 540, var"#9#15"()), Dual{Float64}(0.23300499117218407, 0.0, Dual{Float64}(0.48270590546644865, 0.0, 539, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(0.80295161826784, 0.0, Dual{Float64}[Dual{Float64}(0.45064132668432294, 0.0, Dual{Float64}(0.6712982397446927, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.3287017602553073, 0.0, 540, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.35231029158351707, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.003523102915835171, 0.0, Dual{Float64}(-0.059355731954337576, 0.0, Dual{Float64}[Dual{Float64}(0.048689115240599934, 0.0, 541, var"#9#15"()), Dual{Float64}(0.10804484719493751, 0.0, Dual{Float64}(0.3287017602553073, 0.0, 540, var"#9#15"()), var"#bp!#

), Dual{Float64}(5.7059151853639065, 0.0, Dual{Float64}[Dual{Float64}(0.005785488646542175, 0.0, Dual{Float64}(0.07606239968961126, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.9239376003103887, 0.0, 552, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(5.700129696717364, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.05700129696717365, 0.0, Dual{Float64}(-0.23874944390966368, 0.0, Dual{Float64}[Dual{Float64}(0.614911245357656, 0.0, 553, var"#9#15"()), Dual{Float64}(0.8536606892673196, 0.0, Dual{Float64}(0.9239376003103887, 0.0, 552, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(12.261341747823847, 0.0, Dual{Float64}[Dual{Float64}(0.14829334895199145, 0.0, Dual{Float64}(0.38508875464234404, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0

Excessive output truncated after 524289 bytes.

Float64}(15.495514550567416, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.15495514550567416, 0.0, Dual{Float64}(-0.3936434243140283, 0.0, Dual{Float64}[Dual{Float64}(0.5725098454907389, 0.0, 565, var"#9#15"()), Dual{Float64}(0.9661532698047672, 0.0, Dual{Float64}(0.9829309588189636, 0.0, 564, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMinus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(10.410094298890945, 0.0, Dual{Float64}[Dual{Float64}(0.1827478322023519, 0.0, Dual{Float64}(0.4274901545092611, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.5725098454907389, 0.0, 565, var"#9#15"())], backpropagationForMinus!), var"#bp!#26"{Int64}(2)), Dual{Float64}(10.227346466688592, 0.0, Dual{Float64}[Dual{Float64}(100.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.10227346466688593, 0.0, Dual{Float64}(0.3198022274263985, 0.0

BenchmarkTools.Trial: 
  memory estimate:  103.56 MiB
  allocs estimate:  3887683
  --------------
  minimum time:     4.120 s (0.19% GC)
  median time:      4.124 s (0.19% GC)
  mean time:        4.124 s (0.19% GC)
  maximum time:     4.129 s (0.19% GC)
  --------------
  samples:          2
  evals/sample:     1

In [32]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
@benchmark J(f,6,[rand(3,1)...]) 

1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.8552438755086538, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(2.5903913516569763, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.6475978379142441, 0.0, Dual{Float64}(0.8047346381971165, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.7104877510173075, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.8552438755086538, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0.8552438755086538, 0.0, 3, var"#9#15"()), Dual{Float64}(0.7614155015658037, 0.0, Dual{Float64}(0.865493849748957, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(0.865493849748957, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(2.376179271801362, 0.0, Dual{Float64}(0.865493849748957, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(2.376179271801362)), Dual{Float6

Dual{Float64}[Dual{Float64}(0.11479275939418701, 0.0, Dual{Float64}[Dual{Float64}(0.11953759006136724, 0.0, Dual{Float64}(0.34574208604300294, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.004744830667180227, 0.0, Dual{Float64}(0.16803776866242126, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.06956386847067608, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.653359441274285, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16333986031857126, 0.0, Dual{Float64}(0.404153263402105, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.13912773694135216, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.06956386847067608, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Fl

Dual{Float64}[Dual{Float64}(0.6065000042226879, 0.0, Dual{Float64}[Dual{Float64}(0.6120577739617064, 0.0, Dual{Float64}(0.7823412132578127, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.005557769739018397, 0.0, Dual{Float64}(0.17713328761555314, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.016448616625440282, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.0074930115341688705, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.0018732528835422176, 0.0, Dual{Float64}(0.04328109152438531, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.032897233250880564, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.016448616625440282, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}

Dual{Float64}[Dual{Float64}(0.013380098405767697, 0.0, Dual{Float64}[Dual{Float64}(0.021270637931859734, 0.0, Dual{Float64}(0.14584456771460408, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.007890539526092036, 0.0, Dual{Float64}(0.1990836371925333, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.72304101203325, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(2.5753353317550562, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.6438338329387641, 0.0, Dual{Float64}(0.8023925678486585, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.4460820240665, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.72304101203325, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0

Dual{Float64}[Dual{Float64}(0.3822520889654438, 0.0, Dual{Float64}[Dual{Float64}(0.9969176979875285, 0.0, Dual{Float64}(0.9984576595867891, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.6146656090220847, 0.0, Dual{Float64}(0.850249342776674, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16527921272174861, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(0.4602355294266613, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.11505888235666532, 0.0, Dual{Float64}(0.339203305344546, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.33055842544349723, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.16527921272174861, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}

...

Dual{Float64}[Dual{Float64}(0.015149884127265617, 0.0, Dual{Float64}[Dual{Float64}(0.1834394767445135, 0.0, Dual{Float64}(0.4282983501538542, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.16828959261724788, 0.0, Dual{Float64}(0.5521017025129564, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.8882215714062465, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(3.279839462297879, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.8199598655744698, 0.0, Dual{Float64}(0.9055163530132793, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.776443142812493, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.8882215714062465, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(

Dual{Float64}[Dual{Float64}(0.5789462263619016, 0.0, Dual{Float64}[Dual{Float64}(0.8775439564099148, 0.0, Dual{Float64}(0.9367731616618373, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(0.2985977300480132, 0.0, Dual{Float64}(0.6683882921617701, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(3))], backpropagationForMinus!), Dual{Float64}(88.0, 0.0, nothing, var"#3#4"())]
1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.525440803541795, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(2.298186602273377, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.5745466505683442, 0.0, Dual{Float64}(0.757988555697475, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(1.05088160708359, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.525440803541795, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0.52544

Excessive output truncated after 524293 bytes.

backpropagationForMultiplication!), Dual{Float64}(1.5639231891243068, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(0.7819615945621534, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(0.7819615945621534, 0.0, 3, var"#9#15"()), Dual{Float64}(0.6019859841289051, 0.0, Dual{Float64}(0.6459859068431784, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(0.6459859068431784, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(1.9078670815530097, 0.0, Dual{Float64}(0.6459859068431784, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(1.9078670815530097)), Dual{Float64}(1.473258355034694, 0.0, Dual{Float64}[Dual{Float64}(0.6912967604725406, 0.0, Dual{Float64}[Dual{Float64}(0.6459859068431784, 0.0, 1, var"#9#15"()), Dual{Float64}(0.04531085362936227, 0.0, 2, var"#9#15"())], backpropagationForPlus!), Dual{Float64}(0.7819615945621534, 0.0, 3, var"#9#15"())], backpropagationForPlus!)]
Dual{Float64}[Dual{Float64}(0.4

BenchmarkTools.Trial: 
  memory estimate:  240.70 KiB
  allocs estimate:  8529
  --------------
  minimum time:     2.670 ms (0.00% GC)
  median time:      2.940 ms (0.00% GC)
  mean time:        2.982 ms (0.61% GC)
  maximum time:     6.753 ms (55.08% GC)
  --------------
  samples:          1674
  evals/sample:     1

In [33]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
J(f,6,[2.9,2.2,1546.23]) 


1
Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(1546.23, 0.0, 3, var"#9#15"())]
Dual{Float64}[Dual{Float64}(19.360000000000003, 0.0, Dual{Float64}[Dual{Float64}(4.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(3092.46, 0.0, Dual{Float64}[Dual{Float64}(2.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(1546.23, 0.0, 3, var"#9#15"())], backpropagationForMultiplication!)]
Dual{Float64}[Dual{Float64}(1546.23, 0.0, 3, var"#9#15"()), Dual{Float64}(0.23924932921398243, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(2.9, 0.0, 1, var"#9#15"())))]
Dual{Float64}[Dual{Float64}(18.17414536944306, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#20"{Float64}(18.17414536944306)), Dual{Float64}(1551.33, 0.0, Dual{Float64}[Dual{Float64}(5.1, 0.0, Dual{Float64}[Dual{Float64}(2.9, 0.0, 1, var"#

6-element Array{Array{Float64,2},1}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -2.0913263714842668e-6]
 [0.0; 17.6; -2.0]
 [-1501.3246436992515; 0.0; 0.23924932921398243]
 [0.011707650961926682; -7.55171541667044e-6; -7.55171541667044e-6]
 [5.8; -14.520000000000003; 0.0]

In [34]:
function s(vector)
    vector[1] * vector[2] + sin(vector[1])
end

s (generic function with 1 method)

In [35]:
d = derivativeСalculation(s)

(::var"#dfdx#12"{typeof(s)}) (generic function with 3 methods)

In [36]:
d([1.0, 2.0], 1)

Dual{Float64}[Dual{Float64}(2.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), Dual{Float64}(2.0, 0.0, 2, var"#9#15"())], backpropagationForMultiplication!), Dual{Float64}(0.8414709848078965, 0.0, Dual{Float64}(1.0, 0.0, 1, var"#9#15"()), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(1.0, 0.0, 1, var"#9#15"())))]


2-element Array{Float64,1}:
 2.5403023058681398
 1.0

In [43]:
function styblinskiTang(x)
    value = zero(x[1])
    for i=1:length(x)
        value += x[i]^4 - 16 * x[i]^2 + 5 * x[i]
    end
    value / 2
end

function rosenbrock(x)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

function KarolikJeleniewiczFunction(x)
    f1 = 222.83*(x[2]^8+x[3]^-3)^2 + sin(x[5]^18) - cos(x[1])/exp(x[3])
    f2 = x[1] - 5/x[2] + 4*x[3]^2-2*x[4] / (x[3]*sin(x[5])^2 + 1)
    f3 = sqrt(exp(x[2])^2) + x[3]^x[2] - x[4]
    [f1,f2,f3]
end

function viennet(x)
    f1 = 0.5*(x[1]^2+x[2]^2) + sin(x[1]^2+ x[2]^2)
    f2 = (x[1]-2*x[2]+4)^2/8 + (x[1]-x[2]+1)^2/27 + 15
    f3 = 1/(x[1]^2+x[2]^2+1) - 1.1* exp(-(x[1]^2+x[2]^2))
    [f1,f2,f3]
end


viennet (generic function with 1 method)

In [38]:
d = derivativeСalculation(styblinskiTang)

(::var"#dfdx#12"{typeof(styblinskiTang)}) (generic function with 3 methods)

In [39]:
d([2.9,2.2],1)

Dual{Float64}[Dual{Float64}(-92.34630000000001, 0.0, Dual{Float64}[Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(0.0, 0.0, nothing, var"#5#6"()), Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(-63.831900000000005, 0.0, Dual{Float64}[Dual{Float64}(70.7281, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(4)), Dual{Float64}(134.56, 0.0, Dual{Float64}[Dual{Float64}(16.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForMinus!), Dual{Float64}(14.5, 0.0, Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.9, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(-43.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(-54.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(23.425600000000006, 0.0, Dual{Float64}(2.2, 

2-element Array{Float64,1}:
   4.878
 -11.403999999999996

In [40]:
J(styblinskiTang, 1,[2.9, 2.2])

Dual{Float64}[Dual{Float64}(-92.34630000000001, 0.0, Dual{Float64}[Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(0.0, 0.0, nothing, var"#5#6"()), Dual{Float64}(-49.331900000000005, 0.0, Dual{Float64}[Dual{Float64}(-63.831900000000005, 0.0, Dual{Float64}[Dual{Float64}(70.7281, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(4)), Dual{Float64}(134.56, 0.0, Dual{Float64}[Dual{Float64}(16.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!)], backpropagationForMinus!), Dual{Float64}(14.5, 0.0, Dual{Float64}[Dual{Float64}(5.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.9, 0.0, 1, var"#9#15"())], backpropagationForMultiplication!)], backpropagationForPlus!)], backpropagationForPlus!), Dual{Float64}(-43.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(-54.01440000000001, 0.0, Dual{Float64}[Dual{Float64}(23.425600000000006, 0.0, Dual{Float64}(2.2, 

1-element Array{Array{Float64,2},1}:
 [4.878; -11.403999999999996]

In [41]:
J(viennet, 3, [2.9,2.2])

Dual{Float64}[Dual{Float64}(6.625, 0.0, Dual{Float64}[Dual{Float64}(0.5, 0.0, nothing, var"#1#2"()), Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!)], backpropagationForMultiplication!), Dual{Float64}(0.6316109877182386, 0.0, Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(13.25, 0.0, Dual{Float64}[Dual{Float64}(8.41, 0.0, Dual{Float64}(2.9, 0.0, 1, var"#9#15"()), var"#bp!#26"{Int64}(2)), Dual{Float64}(4.840000000000001, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(2))], backpropagationForPlus!)))]
D

3-element Array{Array{Float64,2},1}:
 [7.39665572674987; 5.611256068568867]
 [0.7509259259259258; -1.3759259259259258]
 [-0.028551403647345923; -0.02165968552557277]

In [47]:
z = J(KarolikJeleniewiczFunction, 3, [2.9,2.2,2.2,34456.5,1.8])

Dual{Float64}[Dual{Float64}(6.712513875660308e7, 0.0, Dual{Float64}[Dual{Float64}(6.712513786463414e7, 0.0, Dual{Float64}[Dual{Float64}(222.83, 0.0, nothing, var"#1#2"()), Dual{Float64}(301239.23109381204, 0.0, Dual{Float64}(548.852649710113, 0.0, Dual{Float64}[Dual{Float64}(548.7587353600004, 0.0, Dual{Float64}(2.2, 0.0, 2, var"#9#15"()), var"#bp!#26"{Int64}(8)), Dual{Float64}(0.0939143501126972, 0.0, Dual{Float64}(0.45454545454545453, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, var"#3#4"()), Dual{Float64}(2.2, 0.0, 3, var"#9#15"())], var"#backpropagationForDividing!#19"{Float64}(0.45454545454545453)), var"#bp!#26"{Int64}(3))], backpropagationForPlus!), var"#bp!#26"{Int64}(2))], backpropagationForMultiplication!), Dual{Float64}(0.891968929252866, 0.0, Dual{Float64}(39346.40807529655, 0.0, Dual{Float64}(1.8, 0.0, 5, var"#9#15"()), var"#bp!#26"{Int64}(18)), var"#bp!#21"{Dual{Float64}}(Dual{Float64}(39346.40807529655, 0.0, Dual{Float64}(1.8, 0.0, 5, var"#9#15"()), var"#bp!#26"{In

3-element Array{Array{Float64,2},1}:
 [0.026509581312979046; 4.880992877171569e8; … ; 0.0; 177883.81382445205]
 [1.0; 1.0330578512396693; … ; -0.6479969546855062; -7042.763658371653]
 [0.0; 13.492961495134765; … ; -1.0; 0.0]

In [48]:
println(z)

[[0.026509581312979046; 4.880992877171569e8; -31325.026289957223; 0.0; 177883.81382445205], [1.0; 1.0330578512396693; 6878.310898259487; -0.6479969546855062; -7042.763658371653], [0.0; 13.492961495134765; 5.666695778750081; -1.0; 0.0]]
